# param 

In [47]:
p_max_depth = 1
p_n_estimators = 2

# import

In [48]:
from google.cloud import bigquery

In [49]:
import pandas as pd

In [50]:
import datetime

In [51]:
import numpy as np

In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [53]:
import pickle as pk

In [54]:
from google.cloud import storage as gcs

In [55]:
import google.cloud.aiplatform as aip

# preprocess

In [56]:
client = bigquery.Client()

In [57]:
def load_from_bq(query):
    return client.query(query).to_dataframe()

In [58]:
query = """
SELECT 
  t1.PassengerId	
  ,Survived	
  ,Pclass		
  ,Name		
  ,Sex
  ,Age	
  ,SibSp
  ,Parch
  ,Ticket
  ,Fare
  ,Cabin
  ,Embarked
FROM `test-hyron.titanic_data.test` t1
LEFT JOIN `test-hyron.titanic_data.gender_submission` t2
  ON t1.PassengerId = t2.PassengerId
"""
df_test = load_from_bq(query)

In [59]:
query = """
SELECT  
    *
FROM `test-hyron.titanic_data.train` 
"""
df_train = load_from_bq(query)

In [60]:
train, val = train_test_split(df_train)

In [61]:
def make_encorder(df):
    x_ = df[["Pclass", "Sex", "SibSp", "Embarked"]]
    
    enc = OneHotEncoder()
    enc.fit(x_)
    
    return enc

In [62]:
enc = make_encorder(df_train)

In [63]:
def make_xy(df, enc):
    x_ = df[["Pclass", "Sex", "Age", "SibSp", "Fare", "Embarked"]]
    
    if "Survived" in df.columns:
        y_ = df["Survived"]
    else:
        y_ = 0
    
    #ここから欠損値の補完やらone-hot化やら
    x_float    = x_[["Age", "Fare"]]
    x_category = x_[["Pclass", "Sex", "SibSp", "Embarked"]]
    
    x_float = x_float.fillna(0)
    
    #enc = OneHotEncoder(handle_unknown='ignore')
    #enc.fit(x_category)
    
    
    
    x = enc.transform(x_category).toarray()

    li = []
    for col_name, elements in zip(x_category.columns, enc.categories_):
        for e in elements:
            li.append(col_name +"_" +  str(e))
    #print(enc.categories_)
    cols_ = list(x_float.columns) + li
    values_ =  np.concatenate([x_float.values, x], axis = 1)
    
    x_ = pd.DataFrame(values_, columns = cols_)
    
    return x_, y_

In [64]:
x_train, y_train = make_xy(train, enc)
x_val, y_val     = make_xy(val, enc)
x_test, y_test        = make_xy(df_test, enc)

# train

In [65]:
clf = RandomForestClassifier(max_depth=p_max_depth , n_estimators = p_n_estimators)

In [66]:
clf.fit(x_train, y_train)

RandomForestClassifier(max_depth=1, n_estimators=2)

# test

In [67]:
val_score = accuracy_score(y_val, clf.predict(x_val))

In [68]:
test_score = accuracy_score(y_test, clf.predict(x_test))

In [69]:
test_score

0.6363636363636364

In [70]:
val_score

0.6098654708520179

# model_save

In [133]:
# client = gcs.Client("test-hyron")
# bucket = client.get_bucket("cyberpot-titanic")
# blob = bucket.blob("model/model.pkl")

In [134]:
# pk.dump(clf, blob.open('wb'))

In [135]:
# model = aip.Model.upload(
#     display_name='my-model',
#     artifact_uri="gs://cyberpot-titanic/model",
#     serving_container_image_uri ="gcr.io/deeplearning-platform-release/sklearn-cpu"
# )